# Simple solution of a modified Eikonal equation
We solve a modified version of the Eikonal equation. The mesh should contain a boundary group labeled `inlet` to locate the inlet boundary condition.

## Import mesh to create topology and geometry
The mesh can be a 1D, 2D or 3D mesh. The elements must be linear and there must be a boundary named `inlet`. 

In [22]:
import os 
import numpy as np

from nutils import function, mesh, solver, export
from nutils.expression_v2 import Namespace

# Nutils namespace
ns = Namespace()

# Geometry
mesh_name = os.path.join("meshes", "gearbox.msh")
domain, geom = mesh.gmsh(mesh_name)

# Define the geometry variable x as well as gradients, normal and jacobians on the domain.
ns.x = geom
ns.define_for("x", gradient="∇", normal="n", jacobians=("dV", "dS"))

gmsh > loaded 3d gmsh topology consisting of #18521 elements
- volume groups: domain #18521
- boundary groups: walls #12146, inlet #44


We sample the domain to get a tuple of all node positions in the mesh `X` and compute a bounding box. The bounding box is then use to define the parameter `l_ref` and consequently the value of $G_0$.

In [23]:
# Get bounding box of input mesh
bezier = domain.sample("vtk", 2) 
X = np.array(bezier.eval(["x_i"] @ ns))
X_min = np.min(X, axis=1)
X_max = np.max(X, axis=1)
l_ref = np.max(X_max-X_min)

# Parameters in namesapce
ns.G0 = 2.0 / l_ref
ns.σ = 0.1

## Basis
A *nutils* basis is a vector-based function that evalautes at any given point $\mathbf{x}$ on the domain to the array of basis functions.

In [24]:
ns.basis = domain.basis("std", degree=1)

## Problem formulation

The problem to be solved is 
$$
\nabla G(\mathbf{x}) \cdot  \nabla  G(\mathbf{x}) + \sigma G(\mathbf{x}) \Delta G(\mathbf{x}) = (1 + 2\sigma)G^4(\mathbf{x}) \quad \mathbf{x} \in \Omega
$$
for the inverse wall distance $G(\mathbf{x})$. The weak form of this equation is
$$
(1-\sigma) \int_\Omega \nabla G(\mathbf{x}) \cdot  \nabla G(\mathbf{x}) H(\mathbf{x}) dV - \sigma \int_V G(\mathbf{x}) \nabla G(\mathbf{x}) \cdot \nabla H(\mathbf{x}) dV - (1+2\sigma) \int_\Omega G^4(\mathbf{x}) H(\mathbf{x}) dV = 0 \quad \forall H
$$
assuming $\nabla G(\mathbf{x}) = 0$ on $\partial \Omega\setminus\partial\Omega_{D}$.

In [25]:
ns.G = function.dotarg("lhs", ns.basis)
res = domain.integral("(1 - σ) ∇_i(G) ∇_i(G) basis_n dV" @ ns, degree=2)
res -= domain.integral("σ G ∇_i(G) ∇_i(basis_n) dV" @ ns, degree=2)
res -= domain.integral("(1 + 2 σ) G^4 basis_n dV" @ ns, degree=2)

## Boundary condition at inlet
The Dirichlet boundary condition is 
$$
G(\mathbf{x}) = G_0 \quad \mathbf{x} \in \partial \Omega_{D}.
$$

It is expressed as 
$$ 
 \underset{\mathbf{G}}{\min} \int_{\partial\Omega} (G - G_0)^2 dS = 0 \quad \text{at} \quad \partial\Omega_\text{D}
$$

In [26]:
sqr = domain.boundary["inlet"].integral("(G - G0)^2 dS" @ ns, degree=2)
inlet = solver.optimize("lhs", sqr, droptol=1e-15)

sqr = domain.boundary["walls"].integral("(G - G0)^2 dS" @ ns, degree=2)
walls = solver.optimize("lhs", sqr, droptol=1e-15)

optimize > solve > solving 36 dof system to machine precision using arnoldi solver
optimize > solve > solver returned with residual 3e-15
optimize > constrained 36/6142 dofs
optimize > optimum value 6.94e-18
optimize > solve > solving 6069 dof system to machine precision using arnoldi solver
optimize > solve > solver returned with residual 9e-14
optimize > constrained 6069/6142 dofs
optimize > optimum value 5.33e-15


## Solve the problem
The problem is actually solve twice: once to determine the distance to inlet and once to determine the distance to all other walls.

In [27]:
# Initial values set to G_0
sqr = domain.integral("(G - G0)^2 dV" @ ns, degree=2)
lhs0 = solver.optimize("lhs", sqr, droptol=1e-15)

# Solve non-linear equations with Newton solvers
lhs_inlet = solver.newton("lhs", res, constrain=inlet, lhs0=lhs0).solve(tol=1E-10)
lhs_walls = solver.newton("lhs", res, constrain=walls, lhs0=lhs0).solve(tol=1E-10)

optimize > solve > solving 6142 dof system to machine precision using arnoldi solver
optimize > solve > solver returned with residual 1e-11
optimize > constrained 6142/6142 dofs
optimize > optimum value 3.41e-13
newton 0% > solve > solving 6106 dof system to tolerance 9e-10 using arnoldi solver
newton 0% > solve > solver returned with residual 2e-20
newton 0% > estimated residual minimum at inf% of update vector
newton 0% > update accepted at relaxation 1.0
newton 12% > solve > solving 6106 dof system to tolerance 3e-10 using arnoldi solver
newton 12% > solve > solver returned with residual 1e-20
newton 12% > estimated residual minimum at 91% of update vector
newton 12% > update accepted at relaxation 1
newton 22% > solve > solving 6106 dof system to tolerance 1e-10 using arnoldi solver
newton 22% > solve > solver returned with residual 4e-21
newton 22% > estimated residual minimum at 94% of update vector
newton 22% > update accepted at relaxation 0.90902
newton 35% > solve > solving 6

## Postprocessing
The resulting vector is interpolated using beziers. The actual distance $D$ is then computed from the inverse wall distance $G$ by
$$
D(\mathbf{x}) = \frac{1}{G(\mathbf{x})} - \frac{1}{G_0}.
$$

In [28]:
ns.D = "1 / G  - 1 / G0"
sample = domain.sample("vtk", None) 
x, D_i = sample.eval(["x_i", "D"] @ ns, lhs=lhs_inlet)
D_w = sample.eval("D" @ ns, lhs=lhs_walls)
export.vtk(mesh_name[:-4], sample.tri, x, D_i=D_i, D_w=D_w)

meshes/gearbox.vtk
